In [1]:
import polars as pl
from pathlib import Path
from pycomfort.files import *
from pycomfort import files
import pyarrow
import pandas as pd
from functional import seq
from typing import *
import functools

In [2]:
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

polars.cfg.Config

In [3]:
gwas = pl.read_csv("/home/antonkulaga/Downloads/gwas-catalog-v1.0.3-studies-r2022-12-15.tsv", sep="\t")
gwas.head()

DATE ADDED TO CATALOG,PUBMED ID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY,SUMMARY STATS LOCATION,SUBMISSION DATE,STATISTICAL MODEL,BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT URI
str,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str
"""2021-03-25""",32701512,"""Cairns J""","""2020-07-23""","""JCI Insight""","""www.ncbi.nlm.nih.gov/pubmed/32701512""","""Pharmacogenomics of aromatase inhibitors in postmenopausal breast cancer and additional mechanisms of anastrozole action.""","""Estrogen level change in anastrozole-treated breast cancer""","""624 individuals""","""NA""","""Illumina [up to 8000000] (imputed)""",5,"""estrogen measurement, response to aromatase inhibitor""","""http://www.ebi.ac.uk/efo/EFO_0011007, http://purl.obolibrary.org/obo/GO_0061477""","""GCST011407""","""Genome-wide genotyping array""",null,null,null,null,"""estrogen-receptor positive breast cancer""","""http://www.ebi.ac.uk/efo/EFO_1000649"""
"""2020-09-15""",32580138,"""Shi XY""","""2020-06-21""","""EBioMedicine""","""www.ncbi.nlm.nih.gov/pubmed/32580138""","""Identification of susceptibility variants to benign childhood epilepsy with centro-temporal spikes (BECTS) in Chinese Han population.""","""Benign childhood epilepsy with centro-temporal spikes""","""1,738 Han Chinese ancestry cases, 6,592 Han Chinese ancestry controls""","""NA""","""Illumina [5352724] (imputed)""",12,"""Rolandic epilepsy""","""http://www.orpha.net/ORDO/Orphanet_1945""","""GCST010566""","""Genome-wide genotyping array""",null,null,null,null,null,null
"""2020-07-09""",32118827,"""Bonde A""","""2020-02-28""","""J Trauma Acute Care Surg""","""www.ncbi.nlm.nih.gov/pubmed/32118827""","""Identification of a new genetic variant associated with cholecystitis: a multicenter genome-wide association study.""","""Acute cholecystitis""","""900 European ancestry cases, 23,727 European ancestry controls""","""NA""","""Illumina [NR] (imputed)""",1,"""Cholecystitis, Acute""","""http://www.ebi.ac.uk/efo/EFO_1001289""","""GCST010176""","""Genome-wide genotyping array""",null,null,null,null,null,null
"""2020-07-09""",32118827,"""Bonde A""","""2020-02-28""","""J Trauma Acute Care Surg""","""www.ncbi.nlm.nih.gov/pubmed/32118827""","""Identification of a new genetic variant associated with cholecystitis: a multicenter genome-wide association study.""","""Cholelithiasis""","""up to 2,377 European ancestry cases, 22,203 European ancestry controls""","""NA""","""Illumina [NR] (imputed)""",0,"""cholelithiasis""","""http://www.ebi.ac.uk/efo/EFO_0004799""","""GCST010177""","""Genome-wide genotyping array""",null,null,null,null,null,null
"""2021-06-22""",33293549,"""Sargurupremraj M""","""2020-12-08""","""Nat Commun""","""www.ncbi.nlm.nih.gov/pubmed/33293549""","""Cerebral small vessel disease genomics and its implications across the lifespan.""","""White matter hyperintensity volume x hypertension interaction (2df)""","""46,055 European ancestry individuals""","""NA""","""Affymetrix, Illumina [9900000] (imputed)""",26,"""hypertension, white matter hyperintensity measurement""","""http://www.ebi.ac.uk/efo/EFO_0000537, http://www.ebi.ac.uk/efo/EFO_0005665""","""GCST011953""","""Genome-wide genotyping array""",null,null,null,null,null,null


In [13]:
traits = gwas\
    .groupby([pl.col("MAPPED_TRAIT").alias("trait"), pl.col("MAPPED_TRAIT_URI").alias("trait_uri")])\
    .agg([pl.count("STUDY ACCESSION").alias("studies"), pl.sum("ASSOCIATION COUNT").alias("association_counts"), pl.count("PUBMED ID").alias("papers")])\
    .sort(["studies","association_counts", "papers"], True)
traits

trait,trait_uri,studies,association_counts,papers
str,str,u32,i64,u32
"""blood protein measurement""","""http://www.ebi.ac.uk/efo/EFO_0007937""",5354,8587,5354
"""protein measurement""","""http://www.ebi.ac.uk/efo/EFO_0004747""",5242,3814,5242
"""metabolite measurement""","""http://www.ebi.ac.uk/efo/EFO_0004725""",3040,2776,3040
"""gut microbiome measurement""","""http://www.ebi.ac.uk/efo/EFO_0007874""",2300,6547,2300
"""functional brain measurement""","""http://www.ebi.ac.uk/efo/EFO_0007849""",1811,147,1811
"""brain connectivity measurement""","""http://www.ebi.ac.uk/efo/EFO_0005210""",1706,100,1706
"""phosphatidylcholine measurement""","""http://www.ebi.ac.uk/efo/EFO_0010226""",733,4805,733
"""clinical treatment""","""http://www.ebi.ac.uk/efo/EFO_0007056""",687,31,687
"""employment status""","""http://www.ebi.ac.uk/efo/EFO_0005241""",687,10,687


In [14]:
traits.write_csv("/home/antonkulaga/Downloads/TRAITS.tsv", sep="\t")

In [ ]:
no_measure = traits.filter(~pl.col("trait").str.contains("measurement") & ~pl.col("trait").str.contains("count"))
    no_measure .write_csv("/home/antonkulaga/Downloads/traits_no_measurement.tsv",  sep="\t")
no_measure#%%
import polars as pl
from pathlib import Path
from pycomfort.files import *
from pycomfort import files
import pyarrow
import pandas as pd
from functional import seq
from typing import *
import functools

In [ ]:
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

In [ ]:
gwas = pl.read_csv("/home/antonkulaga/Downloads/gwas-catalog-v1.0.3-studies-r2022-12-15.tsv", sep="\t")
gwas.head()

In [ ]:
traits = gwas\
    .groupby([pl.col("MAPPED_TRAIT").alias("trait"), pl.col("MAPPED_TRAIT_URI").alias("trait_uri")])\
    .agg([pl.count("STUDY ACCESSION").alias("studies"), pl.sum("ASSOCIATION COUNT").alias("association_counts"), pl.count("PUBMED ID").alias("papers")])\
    .sort(["studies","association_counts", "papers"], True)
traits

In [ ]:
traits.write_csv("/home/antonkulaga/Downloads/TRAITS.tsv", sep="\t")

In [20]:
no_measure = traits.filter(~pl.col("trait").str.contains("measurement") & ~pl.col("trait").str.contains("count"))
no_measure .write_csv("/home/antonkulaga/Downloads/traits_no_measurement.tsv",  sep="\t")
no_measure.shape

(3325, 5)